In [5]:
import os
import requests
from dotenv import load_dotenv
import clickhouse_connect
import itertools
import pandas as pd
import json
load_dotenv()
clickhouse_host = os.getenv("CLICKHOUSE_HOST")
username = os.getenv("CLICKHOUSE_USER")
password = os.getenv("CLICKHOUSE_PASSWORD")
github_token = os.getenv("GITHUB_TOKEN")

client = clickhouse_connect.get_client(host=clickhouse_host, port=8123, username=username, password=password)

headers = {
  "Authorization": f"token {github_token}"
}

In [8]:


repo_names = ["microsoft/graphrag",
"letta-ai/letta",
"FoundationAgents/MetaGPT",
"Significant-Gravitas/AutoGPT",
"eosphoros-ai/DB-GPT",
"deepset-ai/haystack",
"chatchat-space/Langchain-Chatchat",
"openxla/xla",
"searxng/searxng",
"ItzCrazyKns/Perplexica",
"zaidmukaddam/scira",
"google/A2A",
"ComposioHQ/composio",
"songquanpeng/one-api",
"SillyTavern/SillyTavern",
"ChatGPTNextWeb/NextChat",
"chatboxai/chatbox",
"oobabooga/text-generation-webui",
"AUTOMATIC1111/stable-diffusion-webui",
"stackblitz/bolt.new",
"Aider-AI/aider",
"sourcegraph/cody",
"TabbyML/tabby",
"qodo-ai/pr-agent",
"PrefectHQ/prefect",
"Zipstack/unstract",
"iterative/datachain",
"dbt-labs/dbt-core",
"Unstructured-IO/unstructured",
"dask/dask",
"open-compass/opencompass",
"lm-sys/FastChat",
"mannaandpoem/OpenManus",
"camel-ai/owl",
"mindverse/Second-Me",
"NVIDIA/nccl",
"triton-inference-server/server",
"nomic-ai/gpt4all",
"kserve/kserve",
"kvcache-ai/Mooncake",
"vllm-project/aibrix",
"mlc-ai/mlc-llm",
"bentoml/BentoML",
"microsoft/onnxruntime",
"kvcache-ai/ktransformers",
"InternLM/lmdeploy",
"huggingface/text-generation-inference",
"deepflowio/deepflow",
"flyteorg/flyte",
"Netflix/metaflow",
"zenml-io/zenml",
"Farama-Foundation/Gymnasium",
"tensorflow/tensorflow",
"keras-team/keras",
"hpcaitech/ColossalAI",
"microsoft/OmniParser",
"unitycatalog/unitycatalog",
"elastic/elasticsearch",
"opensearch-project/OpenSearch",
"lancedb/lancedb",
"pgvector/pgvector"]

# 通过 GitHub API 获取项目的 star 数据
import requests
import time
from typing import Dict, List

def get_github_repo_stats(repo_name: str, token: str = None) -> Dict:
    """
    获取 GitHub 仓库的统计信息
    
    Args:
        repo_name: 仓库名称，格式为 "owner/repo"
        token: GitHub API token (可选，但建议使用以避免速率限制)
    
    Returns:
        包含仓库统计信息的字典
    """
    url = f"https://api.github.com/repos/{repo_name}"
    headers = {}
    if token:
        headers['Authorization'] = f'token {token}'
    
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            return {
                'repo_name': repo_name,
                'stars': data.get('stargazers_count', 0),
                'forks': data.get('forks_count', 0),
                'watchers': data.get('watchers_count', 0),
                'open_issues': data.get('open_issues_count', 0),
                'created_at': data.get('created_at'),
                'updated_at': data.get('updated_at'),
                'language': data.get('language'),
                'size': data.get('size', 0)
            }
        else:
            print(f"获取 {repo_name} 数据失败: {response.status_code}")
            return None
    except Exception as e:
        print(f"获取 {repo_name} 数据时出错: {e}")
        return None

def batch_get_repo_stats(repo_list: List[str], token: str = None, delay: float = 1.0) -> List[Dict]:
    """
    批量获取多个仓库的统计信息
    
    Args:
        repo_list: 仓库名称列表
        token: GitHub API token
        delay: 请求间隔时间（秒）
    
    Returns:
        包含所有仓库统计信息的列表
    """
    results = []
    total = len(repo_list)
    
    for i, repo_name in enumerate(repo_list, 1):
        print(f"正在获取 {repo_name} 的数据... ({i}/{total})")
        
        stats = get_github_repo_stats(repo_name, token)
        if stats:
            results.append(stats)
        
        # 添加延迟以避免 API 速率限制
        if i < total:
            time.sleep(delay)
    
    return results

# 设置 GitHub API token (建议设置以避免速率限制)
# GITHUB_TOKEN = "your_github_token_here"  # 请替换为您的实际 token
GITHUB_TOKEN = None  # 如果没有 token，设置为 None

# 获取所有项目的统计数据
print("开始获取项目统计数据...")
repo_stats = batch_get_repo_stats(repo_names, token=GITHUB_TOKEN, delay=1.2)

# 转换为 DataFrame
repo_stats_df = pd.DataFrame(repo_stats)
print(f"成功获取 {len(repo_stats_df)} 个项目的数据")

# 显示前几行数据
if not repo_stats_df.empty:
    print("\n项目统计数据预览:")
    print(repo_stats_df[['repo_name', 'stars', 'forks', 'language']].head(10))
    
    # 按 stars 数量排序
    repo_stats_df_sorted = repo_stats_df.sort_values('stars', ascending=False)
    print(f"\nTop 10 最受欢迎的项目 (按 stars 排序):")
    print(repo_stats_df_sorted[['repo_name', 'stars', 'forks']].head(10))
    
    # 保存数据到 CSV 文件
    repo_stats_df.to_csv('data/batch_repo_stats.csv', index=False)
    print("\n数据已保存到 'data/batch_repo_stats.csv'")
else:
    print("未能获取到任何项目数据")




开始获取项目统计数据...
正在获取 microsoft/graphrag 的数据... (1/61)
获取 microsoft/graphrag 数据失败: 403
正在获取 letta-ai/letta 的数据... (2/61)
获取 letta-ai/letta 数据失败: 403
正在获取 FoundationAgents/MetaGPT 的数据... (3/61)
获取 FoundationAgents/MetaGPT 数据失败: 403
正在获取 Significant-Gravitas/AutoGPT 的数据... (4/61)
获取 Significant-Gravitas/AutoGPT 数据失败: 403
正在获取 eosphoros-ai/DB-GPT 的数据... (5/61)
获取 eosphoros-ai/DB-GPT 数据失败: 403
正在获取 deepset-ai/haystack 的数据... (6/61)
获取 deepset-ai/haystack 数据失败: 403
正在获取 chatchat-space/Langchain-Chatchat 的数据... (7/61)
获取 chatchat-space/Langchain-Chatchat 数据失败: 403
正在获取 openxla/xla 的数据... (8/61)
获取 openxla/xla 数据失败: 403
正在获取 searxng/searxng 的数据... (9/61)
获取 searxng/searxng 数据失败: 403
正在获取 ItzCrazyKns/Perplexica 的数据... (10/61)


KeyboardInterrupt: 

In [ ]:
def execute_query_community_openrank(repo_id):
  sql_query = """
  SELECT
      actor_login AS developer_name,
      ROUND(SUM(openrank), 2) AS total_openrank
  FROM
      opensource.community_openrank
  WHERE
      repo_id = %s
  GROUP BY
      actor_login
  ORDER BY
      total_openrank DESC
  """
  formatted_query = sql_query % (f"'{repo_id}'")
  results = client.query(formatted_query)
  return results

# 获取 repo1 开发者数据
repo1_id = '599547518'
repo1_developers = execute_query_community_openrank(repo1_id)
repo1_dev_df = pd.DataFrame(repo1_developers.result_rows, columns=['developer_name', 'total_openrank'])
print(f"repo1 社区开发者总数: {len(repo1_dev_df)} 人")

# 获取 repo2 开发者数据
repo2_id = '740303686'
repo2_developers = execute_query_community_openrank(repo2_id)
repo2_dev_df = pd.DataFrame(repo2_developers.result_rows, columns=['developer_name', 'total_openrank'])
print(f"repo2 社区开发者总数: {len(repo2_dev_df)} 人")

# 查找共同开发者
common_developers = pd.merge(repo1_dev_df, repo2_dev_df, 
                           on='developer_name',
                           suffixes=('_repo1', '_repo2'))
print(f"两个项目的共同开发者总数: {len(common_developers)} 人")

# 查找 OpenRank top 100 中的共同开发者
repo1_top100 = repo1_dev_df.nlargest(200, 'total_openrank')
repo2_top100 = repo2_dev_df.nlargest(200, 'total_openrank')
common_top100 = pd.merge(repo1_top100, repo2_top100,
                        on='developer_name',
                        suffixes=('_vllm', '_sglang'))

print(f"\nOpenRank Top 100 中的共同开发者数量: {len(common_top100)} 人")
print("\nTop 100 共同开发者详情:")
print("开发者名称\t\tvLLM OpenRank\t\tSGLang OpenRank")
print("-" * 60)
for _, row in common_top100.iterrows():
    print(f"{row['developer_name']}\t\t{row['total_openrank_vllm']:.2f}\t\t{row['total_openrank_sglang']:.2f}")

# 将共同开发者数据保存到 CSV 文件
print("\n正在保存共同开发者数据到 overlap_developers.csv...")
common_top100.to_csv('overlap_developers.csv', index=False, encoding='utf-8')
print("数据已保存完成!")


In [ ]:
def execute_query_community_openrank(repo_id):
  sql_query = """
  SELECT 
      DISTINCT issue_author_login AS developer_name,
      COUNT(*) AS pr_count
  FROM
      opensource.events
  WHERE
      repo_id = %s
      AND type = 'PullRequestEvent'
      AND action = 'opened'
  GROUP BY
      issue_author_login
  ORDER BY
      pr_count DESC
  """
  formatted_query = sql_query % (f"'{repo_id}'")
  results = client.query(formatted_query)
  return results

# 获取 repo1 开发者数据
repo1_id = '599547518'
repo1_developers = execute_query_community_openrank(repo1_id)
repo1_dev_df = pd.DataFrame(repo1_developers.result_rows, columns=['developer_name', 'total_openrank'])
print(f"repo1 社区开发者总数: {len(repo1_dev_df)} 人")

# 获取 repo2 开发者数据
repo2_id = '740303686'
repo2_developers = execute_query_community_openrank(repo2_id)
repo2_dev_df = pd.DataFrame(repo2_developers.result_rows, columns=['developer_name', 'total_openrank'])
print(f"repo2 社区开发者总数: {len(repo2_dev_df)} 人")

# 查找共同开发者
common_developers = pd.merge(repo1_dev_df, repo2_dev_df, 
                           on='developer_name',
                           suffixes=('_repo1', '_repo2'))
print(f"两个项目的共同开发者总数: {len(common_developers)} 人")

# 查找 OpenRank top 100 中的共同开发者
repo1_top100 = repo1_dev_df.nlargest(200, 'total_openrank')
repo2_top100 = repo2_dev_df.nlargest(200, 'total_openrank')
common_top100 = pd.merge(repo1_top100, repo2_top100,
                        on='developer_name',
                        suffixes=('_vllm', '_sglang'))

print(f"\nOpenRank Top 100 中的共同开发者数量: {len(common_top100)} 人")
print("\nTop 100 共同开发者详情:")
print("开发者名称\t\tvLLM OpenRank\t\tSGLang OpenRank")
print("-" * 60)
for _, row in common_top100.iterrows():
    print(f"{row['developer_name']}\t\t{row['total_openrank_vllm']:.2f}\t\t{row['total_openrank_sglang']:.2f}")

# 将共同开发者数据保存到 CSV 文件
print("\n正在保存共同开发者数据到 overlap_developers.csv...")
common_top100.to_csv('overlap_developers.csv', index=False, encoding='utf-8')
print("数据已保存完成!")